In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 9, 2021...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 9: Smoke Basin ---
----------

These caves seem to be [lava tubes](https://en.wikipedia.org/wiki/Lava_tube). Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
2199943210
3987894921
9856789892
8767896789
9899965678

```

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the *low points* - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are *four* low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The *risk level* of a low point is *1 plus its height*. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore `*15*`.

Find all of the low points on your heightmap. *What is the sum of the risk levels of all low points on your heightmap?*

In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
print("len:", len(lines))
pprint(lines[:6])

len: 100
['5656921987125678979998760234965456789998768789239876323457896545467894567987232345679209876567998656',
 '4349899876234599568987654345894345999987656678945965212348965434356943459876101956798912987459876541',
 '5498789854345689467898765456789297898999736567899874323489654321248932398765219897987893498598765432',
 '6989598765459789356989878767896345987432125456789985434578965432357953499954398789376789569679876543',
 '9875459876569893249876989898965476976521012345699876645689876543456894689875679643234598998789987656',
 '8764345987678989139765393959796567965432543457789987887792987654569965678976798432123467899899998867']


In [3]:
# Convert to numeric list-of-lists
heightmap = []
for line in lines:
    heightmap.append([int(x) for x in list(line)])


In [4]:
def lookup(heightmap, y, x, missing = 10):
    if y < 0 or y >= len(heightmap):
        return missing 
    if x < 0 or x >= len(heightmap[y]):
        return missing
    else:

        return heightmap[y][x]

answer1 = 0
for y in range(len(heightmap)):
    for x in range(len(heightmap[y])):

        min_height = min(
            lookup(heightmap, y-1, x),
            lookup(heightmap, y+1, x),
            lookup(heightmap, y, x-1),
            lookup(heightmap, y, x+1),
        )
        if heightmap[y][x] < min_height:
            answer1 += 1 + heightmap[y][x]


In [5]:

print("answer 1:", answer1)

answer 1: 452


----

In [6]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 9, 2021...
Saving puzzle description to "README.md"...
Done!



\--- Part Two ---
----------

Next, you need to find the largest basins so you know what areas are most important to avoid.

A *basin* is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The *size* of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

```
2199943210
3987894921
9856789892
8767896789
9899965678

```

The top-right basin, size `9`:

```
2199943210
3987894921
9856789892
8767896789
9899965678

```

The middle basin, size `14`:

```
2199943210
3987894921
9856789892
8767896789
9899965678

```

The bottom-right basin, size `9`:

```
2199943210
3987894921
9856789892
8767896789
9899965678

```

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = *1134*`.

*What do you get if you multiply together the sizes of the three largest basins?*

In [7]:
coords = []
# Convert to list of points with < 9
for y in range(len(heightmap)):
    for x in range(len(heightmap[y])):
        if heightmap[y][x] < 9:
            coords.append((y,x))

basins = []
for p in coords:
    if any([p in basin for basin in basins]):
        # Point exists in basin
        basin_index = [bi for bi in range(len(basins)) if p in basins[bi]][0]
        #print(f"found {p} in basin {basin_index}")
    else:
        # create new basin & add
        basins.append([p])
        basin_index = len(basins) - 1
        #print(f"creating {p} in basin {basin_index}")

    neighbours = [p]
    while len(neighbours) > 0:
        p = neighbours.pop()

        # Add point to current basin, if not already in another
        if not any([p in basin for basin in basins]):
            #print(f"adding {p} to basin {basin_index}")
            basins[basin_index].append(p)

        pn = (p[0]-1, p[1]) # North
        pe = (p[0], p[1]+1) # East
        ps = (p[0]+1, p[1]) # South
        pw = (p[0], p[1]-1) # West

        # Earmark surrounding points 
        for px in [pn, pe, ps, pw]:
            if px not in neighbours and px in coords and not any([px in basin for basin in basins]):
                #print(f"exploring {px}")
                neighbours.append(px)


In [8]:

# Sort largest
answer2 = 1
basin_sizes = [len(basin) for basin in basins]
for i, size in enumerate(reversed(sorted(basin_sizes))):
    print(i, size)
    answer2 *= size
    if i == 2: break


0 115
1 111
2 99


In [9]:
print("answer 2:", answer2)

answer 2: 1263735
